In [ ]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from src.utils import rank_logs
from src.stats import friedman_statistics, iman_davenport, eval_significance, critical_difference_nemenyi, critical_difference_bonferroni_dunn
from src.plot import draw_nemenyi_diagram, draw_bonferroni_dunn_diagram

In [ ]:
with open("./logs/gradual.pkl", "rb") as file:
    logs = pickle.load(file)

In [ ]:
models = {log.model: 0 for log in logs}
synth_datasets = {log.dataset: 0 for log in logs}
N_MODELS = len(models)
N_DATASETS = len(synth_datasets)
print(f"""
MODELS: {N_MODELS}
N_DATASETS: {N_DATASETS}
""")

In [ ]:
params = ["performance", "memory", "time"]
results = []
for param in params:
    friedman_matrix = rank_logs(logs, param, models.keys(), synth_datasets.keys())
    results.append(friedman_matrix.rank(axis=1, ascending=True, method="average").mean())

results = pd.DataFrame(results, index=params)
stats_results = results.copy()
stats_results["fried_stats"] = results.apply(lambda x: friedman_statistics(x, N_DATASETS), axis=1)
stats_results["iman_davenport"] = stats_results["fried_stats"].apply(lambda x: iman_davenport(x, N_DATASETS, N_MODELS))
stats_results["significant"] = stats_results["iman_davenport"].apply(lambda x: eval_significance(x, N_MODELS, N_DATASETS, 0.05))

In [ ]:
rank_logs(logs, "time", models.keys(), synth_datasets.keys())/1e6

In [ ]:
stats_results

In [ ]:
cd = critical_difference_nemenyi(N_MODELS, N_DATASETS, significance=0.05)
for param in params:
    fig = draw_nemenyi_diagram(results.loc[param], cd, title=param.upper())
    plt.show()

In [ ]:
from src.plot import Plots

In [ ]:
plotter = Plots()
plotter.plot_all(logs)